SQLAlchemy のネストされた取得結果を Pydantic 型に cast できるようにする

In [1]:
# 自動生成されたモデル定義

from datetime import datetime
from uuid import UUID

from pydantic import BaseModel, ConfigDict

from tables import UserMeta


class User(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID
    created_at: datetime
    updated_at: datetime
    name: str
    meta: UserMeta
    articles: list[Article] | None = None
    comments: list[Comment] | None = None


class Article(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID
    created_at: datetime
    updated_at: datetime
    author_id: UUID
    title: str
    body: str
    published_at: datetime | None
    author: User | None = None
    comments: list[Comment] | None = None


class Comment(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID
    created_at: datetime
    updated_at: datetime
    article_id: UUID
    author_id: UUID
    body: str
    article: Article | None = None
    author: User | None = None


In [2]:
from sqlalchemy import select
from sqlalchemy.orm import selectinload
from tables import ArticleTable, CommentTable, UserTable

from database import Session, engine

from utils import sa_to_dict

stmt = (
    select(ArticleTable)
    .where(ArticleTable.published_at.is_not(None))
    .order_by(ArticleTable.created_at.desc())
    .options(
        selectinload(ArticleTable.author),
        selectinload(ArticleTable.comments).selectinload(CommentTable.author),
    )
)

with Session(engine) as session:
    article = sa_to_dict(session.scalars(stmt).first())

article

{'id': UUID('225d3f58-131c-4a6b-912c-332135b28749'),
 'created_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
 'updated_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
 'author_id': UUID('098e9ed9-6d0f-4555-a869-fd97abc10eb4'),
 'title': 'Hello, SQLAlchemy 2.x',
 'body': 'This is a dummy article about SQLAlchemy 2.x typed ORM.',
 'published_at': datetime.datetime(2025, 12, 19, 14, 18, 1, 949022, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')),
 'author': {'id': UUID('098e9ed9-6d0f-4555-a869-fd97abc10eb4'),
  'created_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
  'updated_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
  'name': 'Alice',
  'meta': {}},
 'comments': [{'id': UUID('248cb544-3a07-4462-819b-06a34e9b1649'),
   'created_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
   'updated_at': datetime.datetime(2025, 12, 21, 14, 18, 1, 947115),
   'article_id': UUID('225d3f58-131c-4a6b-912c-332135b28749'),
   'author_id': UUID('f14a938b-aa89-48

この articles を Pydantic にマッピングする

In [3]:
Article.model_validate(article)

Article(id=UUID('225d3f58-131c-4a6b-912c-332135b28749'), created_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), updated_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), author_id=UUID('098e9ed9-6d0f-4555-a869-fd97abc10eb4'), title='Hello, SQLAlchemy 2.x', body='This is a dummy article about SQLAlchemy 2.x typed ORM.', published_at=datetime.datetime(2025, 12, 19, 14, 18, 1, 949022, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')), author=User(id=UUID('098e9ed9-6d0f-4555-a869-fd97abc10eb4'), created_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), updated_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), name='Alice', meta={}, articles=None, comments=None), comments=[Comment(id=UUID('248cb544-3a07-4462-819b-06a34e9b1649'), created_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), updated_at=datetime.datetime(2025, 12, 21, 14, 18, 1, 947115), article_id=UUID('225d3f58-131c-4a6b-912c-332135b28749'), author_id=UUID('f14a938b-aa89-48af-a436-a91050e97fbb'), body='yayayay